In [68]:
import pandas as pd
import json
from tqdm import tqdm
import numpy as np
from joblib import dump, load

In [69]:
from openai.embeddings_utils import get_embedding
engine = 'text-embedding-ada-002'
len(get_embedding('text', engine=engine))

1536

# get query embeddings

In [3]:
df_exist = pd.concat([
    pd.read_json('/workspaces/multitask-llm-rnd/datasets/data/query_attr_extract_label/processed2/appen_020323_delivered_022123_validated_query_attr_train_oaiemb.json', lines=True),
    pd.read_json('/workspaces/multitask-llm-rnd/datasets/data/query_attr_extract_label/processed2/appen_020323_delivered_022123_validated_query_attr_val_oaiemb.json', lines=True),
    pd.read_json('/workspaces/multitask-llm-rnd/datasets/data/query_attr_extract_label/processed2/appen_020323_delivered_022123_validated_query_attr_test_oaiemb.json', lines=True)
])

In [4]:
df_train = pd.read_json('/workspaces/multitask-llm-rnd/datasets/data/query_attr_extract_label/processed3/appen_all_validated_query_attr_train.json', lines=True)
df_val = pd.read_json('/workspaces/multitask-llm-rnd/datasets/data/query_attr_extract_label/processed3/appen_all_validated_query_attr_val.json', lines=True)
df_test = pd.read_json('/workspaces/multitask-llm-rnd/datasets/data/query_attr_extract_label/processed3/appen_all_validated_query_attr_test.json', lines=True)

In [5]:
all_queries = set(df_train['query']).union(set(df_val['query'])).union(set(df_test['query']))

In [6]:
len(set(df_exist['query'])), len(all_queries)

(44197, 324005)

In [7]:
df_emb_rest = pd.read_json('appen_tolabel_query_attr_leftover.json', lines=True)

In [8]:
df_embs = pd.concat([
    df_exist[['query', 'openai_embedding']],
    df_emb_rest[['query', 'openai_embedding']]
]).drop_duplicates('query')

In [9]:
len(df_embs) / len(all_queries)

1.543198407431984

In [10]:
df_train_emb = df_train.merge(df_embs, on='query', how='left')
df_val_emb = df_val.merge(df_embs, on='query', how='left')
df_test_emb = df_test.merge(df_embs, on='query', how='left')

In [11]:
len(df_train_emb), len(df_train), len(df_val_emb), len(df_val), len(df_test_emb), len(df_test)

(259204, 259204, 32400, 32400, 32401, 32401)

In [12]:
df_train_emb.loc[df_train_emb.openai_embedding.isna(), 'openai_embedding'] = df_train_emb.loc[df_train_emb.openai_embedding.isna(), 'query'].apply( 
    lambda x: get_embedding(x, engine=engine))
df_val_emb.loc[df_val_emb.openai_embedding.isna(), 'openai_embedding'] = df_val_emb.loc[df_val_emb.openai_embedding.isna(), 'query'].apply( 
    lambda x: get_embedding(x, engine=engine))
df_test_emb.loc[df_test_emb.openai_embedding.isna(), 'openai_embedding'] = df_test_emb.loc[df_test_emb.openai_embedding.isna(), 'query'].apply( 
    lambda x: get_embedding(x, engine=engine))

In [13]:
len(df_train_emb[df_train_emb.openai_embedding.isna()]), len(df_val_emb[df_val_emb.openai_embedding.isna()]), len(df_test_emb[df_test_emb.openai_embedding.isna()])

(0, 0, 0)

In [14]:
df_train_emb.to_json('/workspaces/multitask-llm-rnd/datasets/data/query_attr_extract_label/processed3/appen_all_validated_query_attr_train_oaiemb.json', 
    lines=True, orient='records')
df_val_emb.to_json('/workspaces/multitask-llm-rnd/datasets/data/query_attr_extract_label/processed3/appen_all_validated_query_attr_val_oaiemb.json', 
    lines=True, orient='records')
df_test_emb.to_json('/workspaces/multitask-llm-rnd/datasets/data/query_attr_extract_label/processed3/appen_all_validated_query_attr_test_oaiemb.json', 
    lines=True, orient='records')

# eval simple query model

In [70]:
# df_train_emb = pd.read_json('/workspaces/multitask-llm-rnd/datasets/data/query_attr_extract_label/processed3/appen_all_validated_query_attr_train_oaiemb.json', 
#     lines=True)
# df_val_emb = pd.read_json('/workspaces/multitask-llm-rnd/datasets/data/query_attr_extract_label/processed3/appen_all_validated_query_attr_val_oaiemb.json', 
#     lines=True)
df_test_emb = pd.read_json('/workspaces/multitask-llm-rnd/datasets/data/query_attr_extract_label/processed3/appen_all_validated_query_attr_test_oaiemb.json', 
    lines=True)

In [71]:
# df_train_emb.loc[df_train_emb['attr_name_value_pairs_normalized_text'].apply(lambda x: len(x) == 0), 'attr_name_value_pairs_normalized_text'] = 'unknown'
# df_val_emb.loc[df_val_emb['attr_name_value_pairs_normalized_text'].apply(lambda x: len(x) == 0), 'attr_name_value_pairs_normalized_text'] = 'unknown'
df_test_emb.loc[df_test_emb['attr_name_value_pairs_normalized_text'].apply(lambda x: len(x) == 0), 'attr_name_value_pairs_normalized_text'] = 'unknown'

In [72]:
# X_train = np.array(df_train_emb['openai_embedding'].to_list())
# X_val = np.array(df_val_emb['openai_embedding'].to_list())
X_test = np.array(df_test_emb['openai_embedding'].to_list())

In [73]:
# X_train.shape, X_val.shape, 
X_test.shape

(32401, 1536)

In [74]:
label2id = {}
with open('../../data/attribute_extraction_metadata_template/25L2_unfreetext_attribute_name_value_pairs_02232023.txt', 'r') as f:
    for l in f:
        i = l.replace('\n', '')
        if len(i) > 0:
            label2id[i] = len(label2id)
label2id['unknown'] = len(label2id)

In [75]:
len(label2id)

6876

In [76]:
id2label = {label2id[i]: i for i in label2id}

In [77]:
# y_train = np.zeros((len(X_train), len(label2id)))
# y_val = np.zeros((len(X_val), len(label2id)))
y_test = np.zeros((len(X_test), len(label2id)))

In [78]:
# for ind, i in enumerate(df_train_emb['attr_name_value_pairs_normalized_text'].to_list()):
#     for j in i.split('\n'):
#         if j in label2id:
#             y_train[ind, label2id[j]] = 1

# for ind, i in enumerate(df_val_emb['attr_name_value_pairs_normalized_text'].to_list()):
#     for j in i.split('\n'):
#         if j in label2id:
#             y_val[ind, label2id[j]] = 1

for ind, i in enumerate(df_test_emb['attr_name_value_pairs_normalized_text'].to_list()):
    for j in i.split('\n'):
        if j in label2id:
            y_test[ind, label2id[j]] = 1

In [79]:
# y_train.sum(1).mean(), y_val.sum(1).mean(), 
y_test.sum(1).mean()

1.0771581124039382

In [80]:
y_test.sum(1).min(), y_test.sum(1).max()

(1.0, 9.0)

In [81]:
from sklearn.metrics import (classification_report, accuracy_score, 
    label_ranking_average_precision_score)

In [82]:
clf = load('simple_models/query_attrkv_clf_oaiemb_logistic_v3.joblib')

In [83]:
probas = clf.predict_proba(X_test)

In [84]:
label_ranking_average_precision_score(y_test, probas)

0.7510273384810208

In [85]:
preds = clf.predict(X_test)

In [86]:
report = classification_report(y_test, preds, zero_division=0)
print(report[-500:])

00         0
        6871       0.00      0.00      0.00         0
        6872       0.00      0.00      0.00         1
        6873       0.00      0.00      0.00         0
        6874       0.00      0.00      0.00        43
        6875       0.72      0.78      0.75     17275

   micro avg       0.72      0.51      0.60     34901
   macro avg       0.01      0.00      0.00     34901
weighted avg       0.58      0.51      0.52     34901
 samples avg       0.54      0.53      0.53     34901



In [87]:
report = classification_report(y_test, probas > 0.2, zero_division=0)
print(report[-500:])

00         0
        6871       0.00      0.00      0.00         0
        6872       0.00      0.00      0.00         1
        6873       0.00      0.00      0.00         0
        6874       0.38      0.42      0.40        43
        6875       0.60      0.97      0.74     17275

   micro avg       0.59      0.69      0.64     34901
   macro avg       0.01      0.01      0.01     34901
weighted avg       0.51      0.69      0.57     34901
 samples avg       0.63      0.72      0.65     34901



In [88]:
report = classification_report(y_test, probas > 0.1, zero_division=0)
print(report[-500:])

00         0
        6871       0.00      0.00      0.00         0
        6872       0.00      0.00      0.00         1
        6873       0.00      0.00      0.00         0
        6874       0.33      0.67      0.45        43
        6875       0.56      0.99      0.72     17275

   micro avg       0.53      0.75      0.62     34901
   macro avg       0.01      0.01      0.01     34901
weighted avg       0.47      0.75      0.56     34901
 samples avg       0.60      0.77      0.65     34901



In [89]:
res = []
for ind, i in enumerate((-probas).argsort(1)[:,:10]):
    res_i = []
    for j in i:
        res_i.append((id2label[j], probas[ind, j]))
    res.append(res_i)

In [90]:
df_test_emb['top_preds'] = res

In [101]:
df_test_emb[['query', 'attr_name_value_pairs_normalized', 'top_preds']].sample(5).to_dict('records')

[{'query': 'casa para armar',
  'attr_name_value_pairs_normalized': [],
  'top_preds': [('unknown', 0.8699230375676457),
   ('Materials|Wood', 0.030229983068668736),
   ('Installation or Mount Compatibility|Wall Mount', 0.007058864951471235),
   ('Theme|Home', 0.006994129593506658),
   ('Primary Color|Brown', 0.006691190845992135),
   ('Primary Color|Multicolor', 0.0062792994298387856),
   ('Primary Color|Black', 0.005922252810004793),
   ('Primary Color|Blue', 0.0032515932362382313),
   ('Styles|Art Deco', 0.0032173978764749016),
   ('Primary Color|White', 0.003055567604436222)]},
 {'query': 'throw throw burrito',
  'attr_name_value_pairs_normalized': [['Included Components',
    'Throw Pillow']],
  'top_preds': [('unknown', 0.931902528310769),
   ('Primary Color|Brown', 0.005862086043767474),
   ('Primary Color|Multicolor', 0.005372725447885591),
   ('Theme|TV & Movies', 0.004054119246258599),
   ('Theme|Animals', 0.003709178680723396),
   ('Primary Color|Red', 0.003595764513734315),

In [105]:
emb_i = np.array([get_embedding('red xl shirt disney', engine=engine)])

In [106]:
probas_ = clf.predict_proba(emb_i)
res = []
for ind, i in enumerate((-probas_).argsort(1)[:,:10]):
    res_i = []
    for j in i:
        res_i.append((id2label[j], probas_[ind, j]))
    res.append(res_i)

In [107]:
res

[[('Primary Color|Red', 0.33928501312759113),
  ('unknown', 0.20941147267689375),
  ('Theme|Cartoon', 0.15011342953581217),
  ('Primary Color|Blue', 0.05548318955119273),
  ('Pattern|Cartoon', 0.03804950395816539),
  ('Theme|Cartoons', 0.027971773440853016),
  ('Primary Color|Pink', 0.02222330143135701),
  ('Theme|TV & Movies', 0.02151445237910148),
  ('Primary Color|Yellow', 0.01639633551160718),
  ('Primary Color|Purple', 0.009825727669336438)]]

# get erp embedding

In [3]:
import dvc.api

In [98]:
# df_erp_train = pd.read_json(dvc.api.get_url(
#     'datasets/data/offshore_attr_extract/excel/erp/processed/offshore_amazon_erp_attr_train.json',
#     repo='git@github.com:ContextLogic/multitask-llm-rnd.git'
# ), lines=True)

In [99]:
# df_erp_val = pd.read_json(dvc.api.get_url(
#     'datasets/data/offshore_attr_extract/excel/erp/processed/offshore_amazon_erp_attr_val.json',
#     repo='git@github.com:ContextLogic/multitask-llm-rnd.git'
# ), lines=True)

In [100]:
# df_erp_test = pd.read_json(dvc.api.get_url(
#     'datasets/data/offshore_attr_extract/excel/erp/processed/offshore_amazon_erp_attr_test.json',
#     repo='git@github.com:ContextLogic/multitask-llm-rnd.git'
# ), lines=True)

In [101]:
# len(df_erp_train), len(df_erp_val), len(df_erp_test), 

(39600, 4950, 4951)

In [102]:
# df_erp_train['openai_embedding'] = df_erp_train['query'].apply(lambda x: get_embedding(x, engine=engine))
# df_erp_val['openai_embedding'] = df_erp_val['query'].apply(lambda x: get_embedding(x, engine=engine))
# df_erp_test['openai_embedding'] = df_erp_test['query'].apply(lambda x: get_embedding(x, engine=engine))

In [103]:
# df_erp_train.to_json('/workspaces/multitask-llm-rnd/datasets/data/offshore_attr_extract/excel/erp/processed/offshore_amazon_erp_attr_train_oaiemb.json',  
#     lines=True, orient='records')
# df_erp_val.to_json('/workspaces/multitask-llm-rnd/datasets/data/offshore_attr_extract/excel/erp/processed/offshore_amazon_erp_attr_val_oaiemb.json',  
#     lines=True, orient='records')
# df_erp_test.to_json('/workspaces/multitask-llm-rnd/datasets/data/offshore_attr_extract/excel/erp/processed/offshore_amazon_erp_attr_test_oaiemb.json',  
#     lines=True, orient='records')

# eval erp model

In [108]:
# df_erp_train = pd.read_json('/workspaces/multitask-llm-rnd/datasets/data/offshore_attr_extract/excel/erp/processed/offshore_amazon_erp_attr_train_oaiemb.json', 
#     lines=True)
# df_erp_val = pd.read_json('/workspaces/multitask-llm-rnd/datasets/data/offshore_attr_extract/excel/erp/processed/offshore_amazon_erp_attr_val_oaiemb.json', 
#     lines=True)
df_erp_test = pd.read_json('/workspaces/multitask-llm-rnd/datasets/data/offshore_attr_extract/excel/erp/processed/offshore_amazon_erp_attr_test_oaiemb.json', 
    lines=True)

In [109]:
X_test = np.array(df_erp_test['openai_embedding'].tolist())

In [110]:
y_test = np.zeros((len(X_test), len(label2id)))

In [111]:
df_erp_test.loc[df_erp_test['attr_name_value_pairs_normalized_text'].apply(lambda x: len(x) == 0), 'attr_name_value_pairs_normalized_text'] = 'unknown'

In [112]:
for ind, i in enumerate(df_erp_test['attr_name_value_pairs_normalized_text'].to_list()):
    for j in i.split('\n'):
        if j in label2id:
            y_test[ind, label2id[j]] = 1

In [114]:
y_test.sum(1).mean(), y_test.sum(1).max(), y_test.sum(1).min()

(1.0, 1.0, 1.0)

In [115]:
clf = load('simple_models/query_attrkv_clf_oaiemb_logistic_v3.joblib')

In [134]:
clf_erp = load('simple_models/query_attrkv_clf_oaiemb_logistic_v3_erp.joblib')

In [116]:
emb_erp.shape

(4951, 1536)

In [117]:
probas_ = clf.predict_proba(X_test)
res = []
for ind, i in enumerate((-probas_).argsort(1)[:,:10]):
    res_i = []
    for j in i:
        res_i.append((id2label[j], probas_[ind, j]))
    res.append(res_i)
df_erp_test['top_preds_query'] = res
probas_query_ = probas_

In [135]:
probas_ = clf_erp.predict_proba(X_test)
res = []
for ind, i in enumerate((-probas_).argsort(1)[:,:10]):
    res_i = []
    for j in i:
        res_i.append((id2label[j], probas_[ind, j]))
    res.append(res_i)
df_erp_test['top_preds_erp'] = res
probas_erp_ = probas_

In [141]:
mask = y_test[:,-1] == 0

In [142]:
label_ranking_average_precision_score(y_test[mask], probas_query_[mask])

0.1004638540237284

In [120]:
label_ranking_average_precision_score(y_test, probas_query_)

0.8560293646182596

In [143]:
label_ranking_average_precision_score(y_test[mask], probas_erp_[mask])

0.05040953612194386

In [136]:
label_ranking_average_precision_score(y_test, probas_erp_)

0.8586450874816954

In [122]:
report = classification_report(y_test, probas_query_ > 0.1, zero_division=0)
print(report[-500:])

00         0
        6871       0.00      0.00      0.00         0
        6872       0.00      0.00      0.00         0
        6873       0.00      0.00      0.00         0
        6874       0.00      0.00      0.00         0
        6875       0.85      0.99      0.92      4214

   micro avg       0.79      0.86      0.82      4951
   macro avg       0.00      0.01      0.00      4951
weighted avg       0.73      0.86      0.79      4951
 samples avg       0.83      0.86      0.84      4951



In [144]:
report = classification_report(y_test[mask], probas_query_[mask] > 0.1, zero_division=0)
print(report[-500:])

00         0
        6871       0.00      0.00      0.00         0
        6872       0.00      0.00      0.00         0
        6873       0.00      0.00      0.00         0
        6874       0.00      0.00      0.00         0
        6875       0.00      0.00      0.00         0

   micro avg       0.08      0.09      0.08       737
   macro avg       0.00      0.01      0.00       737
weighted avg       0.06      0.09      0.07       737
 samples avg       0.04      0.09      0.06       737



In [137]:
report = classification_report(y_test, probas_erp_ > 0.1, zero_division=0)
print(report[-500:])

00         0
        6871       0.00      0.00      0.00         0
        6872       0.00      0.00      0.00         0
        6873       0.00      0.00      0.00         0
        6874       0.00      0.00      0.00         0
        6875       0.85      1.00      0.92      4214

   micro avg       0.85      0.85      0.85      4951
   macro avg       0.00      0.00      0.00      4951
weighted avg       0.73      0.85      0.78      4951
 samples avg       0.85      0.85      0.85      4951



In [150]:
report = classification_report(y_test[mask], probas_erp_[mask] > 0.1, zero_division=0)
print(report[-500:])

00         0
        6871       0.00      0.00      0.00         0
        6872       0.00      0.00      0.00         0
        6873       0.00      0.00      0.00         0
        6874       0.00      0.00      0.00         0
        6875       0.00      0.00      0.00         0

   micro avg       0.00      0.00      0.00       737
   macro avg       0.00      0.00      0.00       737
weighted avg       0.00      0.00      0.00       737
 samples avg       0.00      0.00      0.00       737



In [170]:
df_erp_test[(df_erp_test['attr_name_value_pairs_normalized'].apply(len) > 0) & \
    (df_erp_test['top_preds_query'].apply(lambda x: x[0][0] != 'unknown'))][
    ['query', 'attr_name_value_pairs_normalized', 'top_preds_query', 'top_preds_erp']].sample(1).to_dict('records')

[{'query': 'Material Type: PU',
  'attr_name_value_pairs_normalized': [['Materials', 'Polyurethane (PU)']],
  'top_preds_query': [('Materials|Leather', 0.2589480361119265),
   ('unknown', 0.11649041201385812),
   ('Primary Color|Black', 0.012649646896899909),
   ('Primary Color|Purple', 0.00823496060614962),
   ('Primary Color|Brown', 0.0076698606037492265),
   ('Department|Women', 0.007535662501138647),
   ('Primary Color|Multicolor', 0.006786945608152073),
   ('Primary Color|White', 0.006203145084398819),
   ('Primary Color|Blue', 0.0037262290271591826),
   ('Department|Men', 0.0033205296543382585)],
  'top_preds_erp': [('unknown', 0.4277834568938449),
   ('Materials|Leather', 0.0035536975668387064),
   ('Materials|Polyurethane (PU)', 0.0024810873459420184),
   ('Materials|Plastic', 0.0023691858697588086),
   ('Materials|Cotton', 0.0023459189563730005),
   ('Materials|Nylon', 0.0020977730851296267),
   ('Materials|Neoprene', 0.0018711290138382903),
   ('Materials|Suede', 0.0014906740